In [1]:
import pytest
from pydantic import ValidationError

def test_valid_tracking_number():
    tracking = TrackingNumber(value='TH1234567890')
    assert tracking.value == 'TH1234567890'

def test_empty_tracking_number():
    with pytest.raises(ValidationError):
        TrackingNumber(value='')

def test_invalid_type_tracking_number():
    with pytest.raises(ValidationError):
        TrackingNumber(value=12345678)

In [ ]:
from pydantic import BaseModel, Field, field_validator

class TrackingNumber(BaseModel):

    value: str = Field(..., min_length=1, json_schema_extra={'strip_whitespace': True})

    @field_validator('value')
    def check_length(cls, v):
        if len(v) < 5:
            raise ValueError('Tracking Number สั้นเกินไป')
        return v.strip()
    
class Money(BaseModel):
    amount: float = Field(..., ge=0)
    currency: str = Field(..., min_length=3, max_length=3)

    @field_validator('currency')
    def currency_must_be_uppercase(cls, v):
        return v.upper()
    
    def __str__(self):
        return f'{self.amount:,.2f} {self.currency}'


if __name__ == "__main__":
    good_track = TrackingNumber(value='TH1234567890')
    print(f'สำเร็จ: ได้รับพัสดุ {good_track.value}')

    try:
        bad_track = TrackingNumber(value='TH12')
    except ValueError as e:
        print(f'โดนดัก error {e}')

    

สำเร็จ: ได้รับพัสดุ TH1234567890
โดนดัก error 1 validation error for TrackingNumber
value
  Value error, Tracking Number สั้นเกินไป [type=value_error, input_value='TH12', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error


In [4]:
def test_valid_money_creation():
    m = Money(amount=886.26, currency='THB')
    assert m.amount == 886.26
    assert m.currency == 'THB'

def test_money_negative_amount():
    with pytest.raises(ValidationError):
        Money(amount=-100.0, currency='THB')

def test_money_invalid_currency_format():
    with pytest.raises(ValidationError):
        Money(amount=100.0, currenry='THAI_BAHT')
